In [1]:
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Assignment 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .getOrCreate()
)

22/09/13 19:59:37 WARN Utils: Your hostname, U8JUX resolves to a loopback address: 127.0.1.1; using 172.21.180.32 instead (on interface eth0)
22/09/13 19:59:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/13 19:59:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import pandas as pd
from pyspark.sql.functions import *
import numpy as np
import csv
import json

### Postcode

In [28]:
# we extract postcode in Victoria only
tmp_lst = []
with open('../data/raw/postcode.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        tmp_lst.append(row)
df = pd.DataFrame(tmp_lst[1:], columns=tmp_lst[0])

df = df[df['state']=='VIC']

# we select some important columns for this dataset, such as SA2 code, SA2 name, latitude, longitude and so on
data = df[['id','postcode','locality','state','long','lat','SA2_MAINCODE_2016','SA2_NAME_2016']] 

# remove duplicates based on SA2 code and SA2 name
data = data.drop_duplicates(['SA2_MAINCODE_2016','SA2_NAME_2016'])

data.to_csv('../data/curated/postcode_vic.csv')

### Property

In [3]:
# for property json file, we want to convert it into data frame and save as csv
f = open('../data/raw/example.json')
data = json.load(f)
df = pd.DataFrame.from_dict(data, orient='index')
df.to_csv('../data/curated/property.csv')

### Unemployment

In [47]:
# we want information in Victoria only and only analyse unemployment rate

tmp_lst = []
with open('../data/raw/unemployment.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        tmp_lst.append(row)
df = pd.DataFrame(tmp_lst[2:], columns=tmp_lst[1])

df = df[df['Data Item'] == 'Smoothed unemployment rate (%)']

vic_code = []
with open('../data/curated/postcode_vic.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        vic_code.append(row[7])
vic_code = vic_code[1:]

new = df['SA2 Code (2016 ASGS)'].isin(vic_code)
data = df[new]

data.to_csv('../data/curated/unemployment.csv')


### School

In [58]:
tmp_lst = []
with open('../data/raw/school.csv', mode='r', encoding = "ISO-8859-1") as f:
    reader = csv.reader(f)
    header = next(reader)
    for row in reader:
        tmp_lst.append(row)
df = pd.DataFrame(tmp_lst, columns=header)

df = df[['Education_Sector','School_Name','School_Type','Address_Town','Address_State','Address_Postcode','X','Y']]

df.to_csv('../data/curated/school.csv')